In [1]:
#include <iostream>

using namespace std;

# lambdas

Lambda are a syntax to construct a _function object_.

## review - function objects

- Consider a simple `struct` or `class` with an `invoke` member function

In [2]:
struct add_n_simple {
    int _n;
    
    int invoke(int x) const { return _n + x; }
};

- `add_n_simple` can be used as a function which adds `n` to any value passed to invoke

In [3]:
add_n_simple add_s{5};
cout << add_s.invoke(10) << "\n";

15


- We can make invocation more natural by providing a user-defined function call operator, `operator()`, instead of `invoke`
- This is known as a _function object type_

In [4]:
struct add_n_function {
    int _n;
        
    int operator()(int x) const { return _n + x; }
};

- Invocation now uses function calling syntax

In [5]:
add_n_function add_f{5};
cout << add_f(10) << "\n";

15


### anatomy of a function object

- _closure type_
```cpp
struct add_n_function {
```
- _captures_
```cpp
    int _n;
```
- _function call operator_
```cpp
    int operator()(int x) const { return _n + x; }
```
- _closure_
```cpp
add_n_function add_f{5};
```

## lambda expression

- A lambda expression is a prvalue, _closure_, of a unique unnamed class type, the _closure type_

In [6]:
{
    
auto add_n_lambda = [_n = 5](int x) -> int { return _n + x; };
cout << add_n_lambda(10) << "\n";
    
}

15


### anatomy of a lambda expression

- _closure type_
```cpp
    decltype(add_n_lambda)
```
- _captures_
```cpp
    [_n = 5]
```
- _function call operator_
```cpp
    (int x) -> int { return _n + x; }
```
- _closure_
```cpp
    add_n_lambda
```

- The argument list and result type are optional
    - missing return type is `auto` (not `void`)

In [7]:
{

int n = 42;
auto return_n = [n]{ return n; };
cout << return_n() << '\n';

}

42


### captures

- capture types are deduced
- `[]` no capture, convertible to function pointer
- `[=]` capture-default any referenced local variable by value (copy)
- `[&]` capture-default any referenced local variable by reference
- `[x]` captures x by value
- `[&x]` captures x by referece
- `[_x = expression]` capture `_x` as the value of expression
- `[=, &x, _x = expression]` defaults can be miexed with other captures


- captured values are fixed when lambda is created, not at invocation

In [8]:
{

int x = 10;
int y = 20;
    
auto ex1 = [=]{ return x + y; };
auto ex2 = [&]{ x *= y; };
auto ex3 = [x, y]{ return x + y; };
auto ex4 = [x, &y]{ y += x; };
auto ex5 = [_x = ex1()]{ return _x; };
    
cout << ex1() << '\n';
ex2();
cout << ex3() << '\n';
ex4();
cout << ex5() << '\n';
    
cout << "x:" << x << '\n';
cout << "y:" << y << '\n';
    
}
    

30
30
30
x:200
y:30


- `[this]` captures current object by reference
- `[*this]` captures current object by-value (C++17)
- `[args...]` capture pack expansion by value
- `[&args...]` capture pack expansion by reference

### function call operator

- Arguments can be declared `auto` to create a _generic lambda_
- Function can be declared mutable (default is const)
- Functions can be _variadic_ using `auto...`
- Function can be declared noexcept

In [9]:
struct add_t {
    template <class T, class U>
    auto operator()(T x, U y) const { return x + y; }
};

add_t add;

In [10]:
cout << add(5, 10.3) << '\n';

15.3


In [11]:
.undo 2

- Generic lambda

In [12]:
auto add = [](auto x, auto y){ return x + y; };

In [13]:
cout << add(5, 10.3) << '\n';

15.3


- `auto&&` is a forwarding-reference

- mutable function object

In [14]:
struct accumulate_t {
    int _value;
    
    int operator()(int x) {
        _value += x;
        return _value;
    }
};

In [15]:
{
    
accumulate_t accumulate{5};

cout << accumulate(10) << '\n';
cout << accumulate(3) << '\n';
    
}

15
18


- mutable lambda

In [16]:
{

auto accumulate = [_value = 5](int x) mutable {
    _value += x;
    return _value;
};

cout << accumulate(10) << '\n';
cout << accumulate(3) << '\n';
    
}

15
18


- variadic function object

In [17]:
struct print_all_t {
    template <class... Args>
    void operator()(const Args&... args) const {
        (void)initializer_list<int>{((cout << args << '\n'),0)...};
    }
};

In [18]:
print_all_t print_all;

print_all(1, 10.5, "Hello!");

1
10.5
Hello!


In [19]:
.undo 2

- variadic lambda

In [20]:
auto print_all = [](const auto&... args) {
    (void)initializer_list<int>{((cout << args << '\n'),0)...};
};

In [21]:
print_all(1, 10.5, "Hello!");

1
10.5
Hello!


### storing lambdas

- `std::function<>` can hold any callable object with a matching signature

In [22]:
struct lambda_member {
    size_t _count = 0;
    
    function<void(const string&)> _lambda = [_n = _count](const auto& x) {
        for(auto n = _n; n != 0; --n) cout << x << '\n';
    };
};

In [23]:
lambda_member object{4};

object._lambda("Hello");

Hello
Hello
Hello
Hello


- you can a new lambda to a `function`

In [24]:
object._lambda = [](const auto& x) {
    cout << "I heard you the first time, " << x << '\n';
};

object._lambda("World!");

I heard you the first time, World!


#### limitations of std::function

- incurs virtual function call overhead and heap allocation for larger objects
- does not maintain generic or variatic lambdas
- does not support mutable lambdas
    - we will develop a solution for this later in the course

#### limitations of lambdas

- lambdas in C++14 can replace all use cases of std::bind(), except moving or forwarding a pack expansion

In [25]:
template <class F, class... Args>
auto bind_once(F&& f, Args&& ...args) {
    return bind([_f = forward<F>(f)](auto& ...args) mutable {
        return move(_f)(move(args)...);
    }, forward<Args>(args)...);
}

unique_ptr<int> sink(unique_ptr<int>&& x) { return move(x); }

In [26]:
auto bound = bind_once(sink, make_unique<int>(42));

cout << *bound() << '\n';

42


- There is a proposal to [allow pack expansion in lambda init-capture](http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2017/p0780r1.html)
- This would allow `bind_once()` to be expressed as:
```cpp
template <class F, class... Args>
auto bind_once(F&& f, Args&& ...args) {
    return [_f = forward<F>(f), _args = forward<Args>(args)...] () mutable {
        return move(_f)(move(_args)...);
    };
}
```

## recommendations

- Lambda expressions are syntactic sugar - don't let them frighten you
- Naming things, in general, is good. Don't start turning everything into an unnamed type
- Lambdas are useful when
    - the function object would not be reused elsewhere
    - the lambda expression is short
    - the lambda expression is bound to, and executed within, the current context (reference captures)
    - the lambda expression represents a _continuation_ of the current function in a different execution context


## homework

- Find a place in your current project for a function is passed to and standard algorithm or container
    - If your current project doesn't use standard algorithms, find a place to use one!
- Replace it with a lambda expression
- Report the example and your thoughts on the [wiki](git.corp.adobe.com/better-code/class/wiki)

- An example...
```cpp
// Find first position, p, in v, where *p < x
auto p = find_if(begin(v), end(v), [&](const auto& a){ return a < x; });
```

